# Загрузка данных

Добавим координаты домов, метро. Расстояния до метро и центра города.

In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import Yandex
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
from time import sleep
from geopy import distance

import pickle

import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('data\cian_clean.csv')
df.head()

,Название,Адрес,Метро,Время до метро,Цена,Цена за квадрат,Общая площадь,Жилая площадь,Площадь кухни,Этаж,...,Мусоропровод,Количество лифтов,Тип дома,Тип перекрытий,Парковка,Отопление,Балкон/лоджия,Отделка,Этажность здания,Кол-во комнат
0,"5-комн. квартира, 140,6 м²","Москва, ЦАО, р-н Тверской, ул. Петровка, 20/1",Трубная,5.0,82000000.0,583215.0,140.6,81.600,15.600,5,...,Нет,1,Кирпичный,Нет информации,Наземная,Центральное,1,Неизвестно,5,5
1,"5-комн. квартира, 200 м²","Москва, ЦАО, р-н Тверской, Тверская ул., 12С7",Пушкинская,3.0,75000000.0,375000.0,200.0,120.000,20.000,6,...,Да,1,Кирпичный,Нет информации,Наземная,Центральное,1,Неизвестно,6,5
2,"4-комн. апартаменты, 207,9 м²","Москва, ЦАО, р-н Тверской, ул. Охотный Ряд, 2",Охотный ряд,2.0,454889566.0,2188021.0,207.9,122.661,35.343,5,...,Да,1,Монолитный,Нет информации,Наземная,Нет информации,1,Неизвестно,15,4
3,"4-комн. апартаменты, 204,6 м²","Москва, ЦАО, р-н Тверской, ул. Охотный Ряд, 2",Охотный ряд,2.0,508714883.0,2486388.0,204.6,120.714,34.782,6,...,Да,1,Монолитный,Нет информации,Наземная,Нет информации,1,Неизвестно,15,4
4,"5-комн. квартира, 264,6 м²","Москва, ЦАО, р-н Тверской, ул. Большая Дмитров...",Театральная,5.0,450000000.0,1700680.0,264.6,156.114,30.000,7,...,Да,1,Монолитный,Железобетонные,Наземная,Центральное,1,Неизвестно,7,5


# Поиск и добавление координат

In [4]:
geolocator = Yandex(api_key="df6b0204-4bd5-42a4-af8a-5c128c3c9834")

In [5]:
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.5)

## Поиск координат метро.

In [6]:
df['Город и метро'] = 'Москва, метро ' + df['Метро']
df['Город и метро'].head()

0        Москва, метро Трубная
1     Москва, метро Пушкинская
2    Москва, метро Охотный ряд
3    Москва, метро Охотный ряд
4    Москва, метро Театральная
Name: Город и метро, dtype: object

In [7]:
df['Город и метро'].nunique()

295

In [8]:
metro = dict()  # Поиск локаций метро.
for met in df['Город и метро'].unique():
    metro[met] = geocode(met)

In [9]:
metro

{'Москва, метро Трубная': Location(метро Трубная, Люблинско-Дмитровская линия, Москва, Россия, (55.767833, 37.622028, 0.0)),
 'Москва, метро Пушкинская': Location(метро Пушкинская, Таганско-Краснопресненская линия, Москва, Россия, (55.765752, 37.6039, 0.0)),
 'Москва, метро Охотный ряд': Location(метро Охотный Ряд, Сокольническая линия, Москва, Россия, (55.756969, 37.615524, 0.0)),
 'Москва, метро Театральная': Location(метро Театральная, Замоскворецкая линия, Москва, Россия, (55.757967, 37.619001, 0.0)),
 'Москва, метро Чеховская': Location(метро Чеховская, Серпуховско-Тимирязевская линия, Москва, Россия, (55.765843, 37.608167, 0.0)),
 'Москва, метро Тверская': Location(метро Тверская, Замоскворецкая линия, Москва, Россия, (55.76446, 37.605948, 0.0)),
 'Москва, метро Улица 1905 года': Location(метро Улица 1905 года, Таганско-Краснопресненская линия, Москва, Россия, (55.764759, 37.561401, 0.0)),
 'Москва, метро Лубянка': Location(метро Лубянка, Сокольническая линия, Москва, Россия, (55

In [12]:
df['metro'] = df['Город и метро'].map(metro)

In [13]:
df['metro'].isna().sum()

0

In [15]:
df['координаты метро'] = df['metro'].apply(  # Создадим новую колонку с координатами метро.
    lambda x: tuple(x.point) if x else None)

In [16]:
# Создадим новую колонку с широтой метро.
df['широта метро'] = df['координаты метро'].apply(lambda x: x[0])

In [17]:
# Создадим новую колонку с долготой метро.
df['долгота метро'] = df['координаты метро'].apply(lambda x: x[1])

## Поиск координат дома.

In [ ]:
address = dict()
for i in range(0, df['Адрес'].nunique()):
    adr = df['Адрес'].unique()[i]
    address[adr] = geocode(adr, timeout=1)

In [21]:
df['address'] = df['Адрес'].map(address)  # Создадим колонку с локацией дома.

In [23]:
df['координаты дома'] = df['address'].apply(  # Создадим колонку с координатами дома.
    lambda x: tuple(x.point) if x else None)

In [24]:
# Создадим колонку с широтой дома.
df['широта дома'] = df['координаты дома'].apply(lambda x: x[0])

In [25]:
# Создадим колонку с долготой дома.
df['долгота дома'] = df['координаты дома'].apply(lambda x: x[1])

In [26]:
lon_center = 37.621031  # Долгота центра города

In [30]:
lat_center = 55.753595  # Широта центра города

## Поиск расстояний до центра и метро.

In [28]:
df['Расстояние до центра'] = df[['широта дома', 'долгота дома']].apply(
    lambda x: distance.distance((x[0], x[1]), (lat_center, lon_center)).km, axis=1)

In [29]:
df['Расстояние до метро'] = df[['широта дома', 'долгота дома', 'широта метро', 'долгота метро']].apply(
    lambda x: distance.distance((x[0], x[1]), (x[2], x[3])).km, axis=1)

# Заполнение пропусков в признаке "время до метро"

In [32]:
man_speed = 0.083  # Средняя скорость ходьбы человека в км/мин

In [62]:
"""Заполним пропуски в признаке 'Время до метро'"""
for idx, name in df[df['Время до метро'].isna()].iterrows():
    speed = (df['Расстояние до метро'][idx] / man_speed).astype(int)
    df['Время до метро'][idx] = speed

In [61]:
df['Время до метро'].isna().sum()

0

In [74]:
df['Время до метро'] = df['Время до метро'].astype(int)

# Добавление признака "Округ"

In [63]:
districts = ["ЦАО", "ЮАО", 'ЮЗАО', 'ЮВАО', 'ЗАО', 'СВАО', 'ВАО', 'САО',
             'СЗАО', 'НАО (Новомосковский)', 'ЗелАО']  # Список всех округов Москвы.

In [64]:
df['Адрес'].str. split(', ', expand=True)[1]

0                         ЦАО
1                         ЦАО
2                         ЦАО
3                         ЦАО
4                         ЦАО
                 ...         
10762    Новоалексеевская ул.
10763                    СЗАО
10764                    ЮЗАО
10765                     ЮАО
10766                     ЮАО
Name: 1, Length: 10767, dtype: object

In [96]:
df['district'] = df['Адрес'].str. split(', ', expand=True)[1]

In [97]:
df['district'].value_counts()

district
ЦАО                      2729
ЮАО                      1444
ЮЗАО                     1183
ЮВАО                     1027
ЗАО                       944
                         ... 
Щербаковская ул.            1
Ордынский туп.              1
ул. 4-я Марьиной рощи       1
Амурская ул.                1
Новоалексеевская ул.        1
Name: count, Length: 115, dtype: int64

In [99]:
district = dict()
for i in range(0, len(df['Метро'])):
    if i not in district.keys():
        if df['district'][i] in districts:
            district[df['Метро'][i]] = df['district'][i]

In [103]:
df['Округ'] = df['Метро'].map(district)

In [104]:
"""Заменим все значения где не указан округ на 'Неизвестно' """

for ind, dis in enumerate(df['Округ']):
    if dis not in districts:
        df['Округ'][ind] = 'Неизвестно'

C:\Users\ser34\AppData\Local\Temp\ipykernel_5636\657050907.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Округ'][ind] = 'Неизвестно'
C:\Users\ser34\AppData\Local\Temp\ipykernel_5636\657050907.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Округ'][ind] = 'Неизвестно'
C:\Users\ser34\AppData\Local\Temp\ipykernel_5636\657050907.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Округ'][ind] = 'Неизвестно'
C:\Users\s

Уберем уточнение "Новомосковский".

In [113]:
df['Округ'] = df['Округ'].str.split(' ', expand=True)[0]

In [114]:
df['Округ'].value_counts()

Округ
ЦАО           2707
ЮАО           1410
ЮЗАО          1401
ЮВАО          1081
ЗАО            962
СВАО           953
ВАО            826
САО            772
СЗАО           515
НАО            129
Неизвестно       6
ЗелАО            5
Name: count, dtype: int64

# Удаление лишних колонок

In [117]:
df_clean = df.drop(['metro', 'address', 'Город и метро',
                    'координаты метро', 'координаты дома', 'district'], axis=1)

In [118]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10767 entries, 0 to 10766
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Название              10767 non-null  object 
 1   Адрес                 10767 non-null  object 
 2   Метро                 10767 non-null  object 
 3   Время до метро        10767 non-null  int32  
 4   Цена                  10767 non-null  float64
 5   Цена за квадрат       10767 non-null  float64
 6   Общая площадь         10767 non-null  float64
 7   Жилая площадь         10767 non-null  float64
 8   Площадь кухни         10767 non-null  float64
 9   Этаж                  10767 non-null  int64  
 10  Год постройки         10767 non-null  int64  
 11  Тип жилья             10767 non-null  object 
 12  Высота потолков       10767 non-null  float64
 13  Санузел               10767 non-null  int64  
 14  Вид из окон           10767 non-null  object 
 15  Ремонт             

In [119]:
df_clean.to_csv("data\cian_full.csv", index=False)